In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
input_file_name = "/Users/abuzatu/Work/data/finance/stocks/Revolut/Revolut_trades_2020.txt"
input_file_name_end = "/Users/abuzatu/Work/data/finance/stocks/Revolut/Revolut_trades_2020_end.txt"
input_file_name_deposits = "/Users/abuzatu/Work/data/finance/stocks/Revolut/Revolut_trades_2020_deposits.txt"

In [ ]:
def read_file(input_file_name):
    file = open(input_file_name, "r")
    lines = file.readlines()
    for line in lines:
        line = line.rstrip()
        # print(line)
        list_el = line.split()
        if len(list_el) != 5:
            print(list_el)
        stock = float(list_el[2])
        price = float(list_el[3])
        cash = float(list_el[4])
        if ((stock * price) + cash) > 0.01:
            print(list_el)
        if ((stock * price) * cash) > 0:
            print(list_el)
        
# read_file(input_file_name)

In [ ]:
def get_date(x):
    list_el = x.split("/")
    return pd.to_datetime(f"2020-{list_el[0]}-{list_el[1]}")

def change_date(df):  
    df["date"] = df.date.map(lambda x: get_date(x))
    df.set_index("date", inplace = True)
    return df

# Deposits

In [ ]:
df_deposit = pd.read_csv(input_file_name_deposits, delimiter=" ")
df_deposit

In [ ]:
df_deposit = change_date(df_deposit)
df_deposit

In [ ]:
df_deposit.deposit.sum()

# Trades

In [ ]:
df = pd.read_csv(input_file_name, delimiter=" ")
df

In [ ]:
# check
df["multiply"] = df["stock"] * df["cash"]
df[df["multiply"] > 0.0]

In [ ]:
# the value cash should be used, as it takes into account also the fees paid for the trade
# for the first few months I paid 1 euro per trade, plus the fee from SEC of 0.01$ per trade
df["net"] = (df["stock"] * df["price"]) + df["cash"]
df.sort_values(by="net")

In [ ]:
# maybe so much used in fees
df.net.sum()

In [ ]:
df.drop(["multiply", "net"], axis = 1, inplace = True)

In [ ]:
df.head(2)

In [ ]:
df = change_date(df)
df

In [ ]:
df.index[0]

In [ ]:
df["counter"] = 1
df

In [ ]:
# group by ticker
df1 = df.groupby("ticker").agg("sum")
df1.drop(["price"], axis = 1, inplace = True)
df1["stock"] = df1["stock"].map(lambda x: round(x, 6))
df1

In [ ]:
# 39 shares traded
len(df1.index.unique())

In [ ]:
df_AAPL = df[df.ticker == "AAPL"]
df_AAPL

In [ ]:
df_AAPL.iloc[0:7]

In [ ]:
df_AAPL.iloc[0:7].cash.sum()

In [ ]:
df_AAPL.iloc[0:7].stock.sum()

In [ ]:
df_TSLA = df[df.ticker == "TSLA"]
df_TSLA[0:100]

In [ ]:
df_TSLA.cash.sum()

In [ ]:
df_TSLA.stock.sum()

In [ ]:
df_9 = df[df.ticker == "MRNA"]
df_9

In [ ]:
df_9.cash.sum()

In [ ]:
df1.loc["ENPH"]

In [ ]:
df1.cash.sum()

# Stocks at end of period

In [ ]:
df_end = pd.read_csv(input_file_name_end, delimiter=" ")
df_end

In [ ]:
df_end["value_end"] = df_end["stock_end"] * df_end["price_end"]
df_end

In [ ]:
df_end.value_end.sum()

In [ ]:
df_end.total_cost.sum()

In [ ]:
len(df1)

In [ ]:
len(df_end)

In [ ]:
df_end

In [ ]:
df2 = df_end.merge(df1, on = "ticker", how = "outer")
df2

In [ ]:
# fill NaN falues with zero
df2.fillna(0.0, inplace = True)
df2

As expected, the values in the stock column are identifical to those in the stock_end, since we add the trades of 2020, which are from the beginning of trading at Revolut to the end. But let's also check explicitely that it is the same for all

In [ ]:
# There should be nothing here
df2[df2["stock_end"] - df2["stock"] > 0.001]

In [ ]:
df2.head(3)

In [ ]:
df2["real_total_cost"] = - df2["cash"]
df2

Stocks still had shown at `stock_end` each at average `unit_cost`, with a `total_cost`, and a the `price_end` produce a `value_end`, where `value_end-total_cost` produces the unrealised gain on the stock

The real total gain 

In [ ]:
df2["unrealised_gain"] = df2["value_end"] - df2["total_cost"]
df2["total_gain"] = df2["value_end"] - df2["real_total_cost"]
df2["realised_gain"] = df2["total_gain"] - df2["unrealised_gain"]
df2["realised_gain2"] = - df2["real_total_cost"] + df2["total_cost"]
# df2["realised_gain"] = df2["value_end"] + df2["cash"]
df2.head(3)

In [ ]:
# for each stock, calculate the value the stock has + the cash I spent on the stock
# to see if overall I am in positive or negative with this stock
df2["final"] = df2["value_end"] + df2["cash"]
# final2 uses the direct cost of buying, not the losses already made in the stock like final does
df2["final2"] = df2["total_cost"] + df2["cash"]
df2["final2"] = df2["final2"].map(lambda x: round(x, 6))
df2

In [ ]:
# sort by final
df2.sort_values(by="final2", inplace = True)
df2

In [ ]:
df2.final.sum()

In [ ]:
df2.final2.sum()

In [ ]:
df2.realised_gain.sum()

In [ ]:
df2.realised_gain2.sum()

In [ ]:
df2.total_gain.sum()

In [ ]:
df2.unrealised_gain.sum()

In [ ]:
df_deposit.deposit.sum()

In [ ]:
df2.value_end.sum()

In [ ]:
df2.cash.sum()

In [ ]:
df2.total_cost.sum()

In [ ]:
df2.value_end.sum() - df_deposit.deposit.sum()

In [ ]:
df2.total_cost.sum() - df_deposit.deposit.sum()

# Conclusion

A realised loss of 96.60 euro was obtained at Revolut in calendar year 2020.